In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset2') 

# Actions that we try to detect
actions = np.array(['hello', 'thankyou', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [9]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'hello': 0, 'thankyou': 1, 'iloveyou': 2}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(90, 30, 1662)

In [16]:
np.array(labels).shape

(90,)

In [17]:
X = np.array(sequences)

In [18]:
X.shape

(90, 30, 1662)

In [19]:
y = to_categorical(labels).astype(int)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [21]:
y_test.shape

(5, 3)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [9]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 824ms/step - loss: 1.9037 - categorical_accuracy: 0.3529
Epoch 2/2000
3/3 [==============================] - 0s 57ms/step - loss: 2.2131 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 52ms/step - loss: 2.5153 - categorical_accuracy: 0.2941
Epoch 4/2000
3/3 [==============================] - 1s 303ms/step - loss: 1.9664 - categorical_accuracy: 0.3529
Epoch 5/2000
3/3 [==============================] - 0s 77ms/step - loss: 3.6808 - categorical_accuracy: 0.3529
Epoch 6/2000
3/3 [==============================] - 0s 134ms/step - loss: 1.4667 - categorical_accuracy: 0.2118
Epoch 7/2000
3/3 [==============================] - 0s 80ms/step - loss: 2.7073 - categorical_accuracy: 0.4353
Epoch 8/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.2430 - categorical_accuracy: 0.4706
Epoch 9/2000
3/3 [==============================] - 1s 219ms/step - loss: 3.9839 - categorical_accuracy: 0.2

3/3 [==============================] - 0s 69ms/step - loss: 1.0923 - categorical_accuracy: 0.3059
Epoch 75/2000
3/3 [==============================] - 0s 66ms/step - loss: 1.0899 - categorical_accuracy: 0.3765
Epoch 76/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.0867 - categorical_accuracy: 0.3647
Epoch 77/2000
3/3 [==============================] - 0s 64ms/step - loss: 1.0812 - categorical_accuracy: 0.5294
Epoch 78/2000
3/3 [==============================] - 0s 79ms/step - loss: 1.0682 - categorical_accuracy: 0.5412
Epoch 79/2000
3/3 [==============================] - 0s 66ms/step - loss: 1.1562 - categorical_accuracy: 0.5176
Epoch 80/2000
3/3 [==============================] - 0s 71ms/step - loss: 2.0893 - categorical_accuracy: 0.3412
Epoch 81/2000
3/3 [==============================] - 0s 74ms/step - loss: 1.1018 - categorical_accuracy: 0.3529
Epoch 82/2000
3/3 [==============================] - 0s 140ms/step - loss: 1.1029 - categorical_accuracy: 0.3529
Epoch

3/3 [==============================] - 0s 65ms/step - loss: 1.7462e-04 - categorical_accuracy: 1.0000
Epoch 218/2000
3/3 [==============================] - 0s 67ms/step - loss: 1.6031e-04 - categorical_accuracy: 1.0000
Epoch 219/2000
3/3 [==============================] - 0s 75ms/step - loss: 1.4753e-04 - categorical_accuracy: 1.0000
Epoch 220/2000
3/3 [==============================] - 0s 71ms/step - loss: 1.5354e-04 - categorical_accuracy: 1.0000
Epoch 221/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.3266e-04 - categorical_accuracy: 1.0000
Epoch 222/2000
3/3 [==============================] - 0s 78ms/step - loss: 1.4724e-04 - categorical_accuracy: 1.0000
Epoch 223/2000
3/3 [==============================] - 0s 83ms/step - loss: 1.2123e-04 - categorical_accuracy: 1.0000
Epoch 224/2000
3/3 [==============================] - 0s 93ms/step - loss: 1.3277e-04 - categorical_accuracy: 1.0000
Epoch 225/2000
3/3 [==============================] - 0s 112ms/step - loss: 1.2

Epoch 287/2000
3/3 [==============================] - 0s 67ms/step - loss: 3.5497e-05 - categorical_accuracy: 1.0000
Epoch 288/2000
3/3 [==============================] - 0s 58ms/step - loss: 3.5111e-05 - categorical_accuracy: 1.0000
Epoch 289/2000
3/3 [==============================] - 0s 67ms/step - loss: 3.4776e-05 - categorical_accuracy: 1.0000
Epoch 290/2000
3/3 [==============================] - 0s 152ms/step - loss: 3.4431e-05 - categorical_accuracy: 1.0000
Epoch 291/2000
3/3 [==============================] - 1s 241ms/step - loss: 3.4117e-05 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 58ms/step - loss: 3.3758e-05 - categorical_accuracy: 1.0000
Epoch 293/2000
3/3 [==============================] - 0s 58ms/step - loss: 3.3459e-05 - categorical_accuracy: 1.0000
Epoch 294/2000
3/3 [==============================] - 0s 67ms/step - loss: 3.3149e-05 - categorical_accuracy: 1.0000
Epoch 295/2000
3/3 [==============================] - 0s 64ms/

Epoch 356/2000
3/3 [==============================] - 0s 60ms/step - loss: 2.1050e-05 - categorical_accuracy: 1.0000
Epoch 357/2000
3/3 [==============================] - 0s 67ms/step - loss: 2.0918e-05 - categorical_accuracy: 1.0000
Epoch 358/2000
3/3 [==============================] - 1s 161ms/step - loss: 2.0800e-05 - categorical_accuracy: 1.0000
Epoch 359/2000
3/3 [==============================] - 1s 142ms/step - loss: 2.0663e-05 - categorical_accuracy: 1.0000
Epoch 360/2000
3/3 [==============================] - 0s 59ms/step - loss: 2.0558e-05 - categorical_accuracy: 1.0000
Epoch 361/2000
3/3 [==============================] - 0s 56ms/step - loss: 2.0418e-05 - categorical_accuracy: 1.0000
Epoch 362/2000
3/3 [==============================] - 0s 60ms/step - loss: 2.0301e-05 - categorical_accuracy: 1.0000
Epoch 363/2000
3/3 [==============================] - 0s 86ms/step - loss: 2.0185e-05 - categorical_accuracy: 1.0000
Epoch 364/2000
3/3 [==============================] - 0s 118ms

...

3/3 [==============================] - 0s 77ms/step - loss: 1.0504e-06 - categorical_accuracy: 1.0000
Epoch 1323/2000
3/3 [==============================] - 0s 67ms/step - loss: 1.0490e-06 - categorical_accuracy: 1.0000
Epoch 1324/2000
3/3 [==============================] - 0s 67ms/step - loss: 1.0490e-06 - categorical_accuracy: 1.0000
Epoch 1325/2000
3/3 [==============================] - 1s 233ms/step - loss: 1.0476e-06 - categorical_accuracy: 1.0000
Epoch 1326/2000
3/3 [==============================] - 0s 125ms/step - loss: 1.0434e-06 - categorical_accuracy: 1.0000
Epoch 1327/2000
3/3 [==============================] - 0s 58ms/step - loss: 1.0406e-06 - categorical_accuracy: 1.0000
Epoch 1328/2000
3/3 [==============================] - 0s 74ms/step - loss: 1.0364e-06 - categorical_accuracy: 1.0000
Epoch 1329/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.0364e-06 - categorical_accuracy: 1.0000
Epoch 1330/2000
3/3 [==============================] - 0s 58ms/step - 

3/3 [==============================] - 0s 74ms/step - loss: 7.6995e-07 - categorical_accuracy: 1.0000
Epoch 1460/2000
3/3 [==============================] - 0s 57ms/step - loss: 7.6715e-07 - categorical_accuracy: 1.0000
Epoch 1461/2000
3/3 [==============================] - 0s 67ms/step - loss: 7.6715e-07 - categorical_accuracy: 1.0000
Epoch 1462/2000
3/3 [==============================] - ETA: 0s - loss: 6.8173e-07 - categorical_accuracy: 1.00 - 0s 58ms/step - loss: 7.6715e-07 - categorical_accuracy: 1.0000
Epoch 1463/2000
3/3 [==============================] - 0s 160ms/step - loss: 7.6715e-07 - categorical_accuracy: 1.0000
Epoch 1464/2000
3/3 [==============================] - 0s 111ms/step - loss: 7.6154e-07 - categorical_accuracy: 1.0000
Epoch 1465/2000
3/3 [==============================] - 0s 100ms/step - loss: 7.6013e-07 - categorical_accuracy: 1.0000
Epoch 1466/2000
3/3 [==============================] - 0s 55ms/step - loss: 7.6013e-07 - categorical_accuracy: 1.0000
Epoch 1467/

3/3 [==============================] - 0s 67ms/step - loss: 5.7641e-07 - categorical_accuracy: 1.0000
Epoch 1596/2000
3/3 [==============================] - 0s 90ms/step - loss: 5.7501e-07 - categorical_accuracy: 1.0000
Epoch 1597/2000
3/3 [==============================] - 0s 91ms/step - loss: 5.7501e-07 - categorical_accuracy: 1.0000
Epoch 1598/2000
3/3 [==============================] - 0s 64ms/step - loss: 5.7361e-07 - categorical_accuracy: 1.0000
Epoch 1599/2000
3/3 [==============================] - 0s 97ms/step - loss: 5.7080e-07 - categorical_accuracy: 1.0000
Epoch 1600/2000
3/3 [==============================] - 0s 59ms/step - loss: 5.7080e-07 - categorical_accuracy: 1.0000
Epoch 1601/2000
3/3 [==============================] - 0s 59ms/step - loss: 5.7080e-07 - categorical_accuracy: 1.0000
Epoch 1602/2000
3/3 [==============================] - 0s 84ms/step - loss: 5.7080e-07 - categorical_accuracy: 1.0000
Epoch 1603/2000
3/3 [==============================] - 0s 63ms/step - lo

3/3 [==============================] - 0s 72ms/step - loss: 4.2495e-07 - categorical_accuracy: 1.0000
Epoch 1734/2000
3/3 [==============================] - 0s 58ms/step - loss: 4.1934e-07 - categorical_accuracy: 1.0000
Epoch 1735/2000
3/3 [==============================] - 0s 134ms/step - loss: 4.1793e-07 - categorical_accuracy: 1.0000
Epoch 1736/2000
3/3 [==============================] - 1s 252ms/step - loss: 4.1793e-07 - categorical_accuracy: 1.0000
Epoch 1737/2000
3/3 [==============================] - 0s 53ms/step - loss: 4.1653e-07 - categorical_accuracy: 1.0000
Epoch 1738/2000
3/3 [==============================] - 0s 53ms/step - loss: 4.1232e-07 - categorical_accuracy: 1.0000
Epoch 1739/2000
3/3 [==============================] - 0s 79ms/step - loss: 4.0812e-07 - categorical_accuracy: 1.0000
Epoch 1740/2000
3/3 [==============================] - 0s 78ms/step - loss: 4.0531e-07 - categorical_accuracy: 1.0000
Epoch 1741/2000
3/3 [==============================] - 1s 384ms/step -

Epoch 1871/2000
3/3 [==============================] - 0s 68ms/step - loss: 2.8891e-07 - categorical_accuracy: 1.0000
Epoch 1872/2000
3/3 [==============================] - 0s 75ms/step - loss: 2.8891e-07 - categorical_accuracy: 1.0000
Epoch 1873/2000
3/3 [==============================] - 0s 112ms/step - loss: 2.8750e-07 - categorical_accuracy: 1.0000
Epoch 1874/2000
3/3 [==============================] - 0s 127ms/step - loss: 2.8610e-07 - categorical_accuracy: 1.0000
Epoch 1875/2000
3/3 [==============================] - 0s 136ms/step - loss: 2.8610e-07 - categorical_accuracy: 1.0000
Epoch 1876/2000
3/3 [==============================] - 0s 66ms/step - loss: 2.8470e-07 - categorical_accuracy: 1.0000
Epoch 1877/2000
3/3 [==============================] - 0s 66ms/step - loss: 2.8189e-07 - categorical_accuracy: 1.0000
Epoch 1878/2000
3/3 [==============================] - 0s 67ms/step - loss: 2.8189e-07 - categorical_accuracy: 1.0000
Epoch 1879/2000
3/3 [==============================] 

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [28]:
res = model.predict(X_test)

In [29]:
actions[np.argmax(res[4])]

'thankyou'

In [30]:
actions[np.argmax(y_test[4])]

'thankyou'

In [31]:
model.save('mod2.h5')

In [11]:
model.load_weights('mod2.h5')

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [34]:

yhat = model.predict(X_test)

In [35]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [37]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[2, 0],
        [0, 3]]], dtype=int64)

In [38]:
accuracy_score(ytrue, yhat)

1.0

In [12]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<